In [46]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
# imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [48]:
# load data
path = '/content/drive/MyDrive/Coding Dojo/Raw Data/insurance.csv'
df = pd.read_csv(path)
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


How well can the charge be predicted based on age, sex, BMI, number of children, smoking habit, and region of the patient?

In [49]:
# define features (X) and target (y)
y = df['charges']
X = df.drop(columns = 'charges')

In [50]:
# train test split to prepare for machine learning
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [51]:
# categorical selector
cat_selector = make_column_selector(dtype_include = 'object')
num_selector = make_column_selector(dtype_include = 'number')

In [52]:
# view categorical columns
cat_data = X_train[cat_selector(X_train)]
cat_data

,sex,smoker,region
693,male,no,northwest
1297,female,no,southeast
634,male,no,southwest
1022,male,yes,southeast
178,female,no,southwest
...,...,...,...
1095,female,no,northeast
1130,female,no,southeast
1294,male,no,northeast
860,female,yes,southwest


In [53]:
num_data = X_train[num_selector(X_train)]
num_data

,age,bmi,children
693,24,23.655,0
1297,28,26.510,2
634,51,39.700,1
1022,47,36.080,1
178,46,28.900,2
...,...,...,...
1095,18,31.350,4
1130,39,23.870,5
1294,58,25.175,0
860,37,47.600,2


The 'age' feature is numerical.

The 'sex' feature is nominal.

The 'bmi' feature is numerical.

The 'children' feature is numerical.

The 'smoker' feature is nominal.

The 'region' feature is nominal.

In [54]:
# encode nominal features using OneHotEncoder
ohe_encoder = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
ohe_encoder.fit(cat_data)
nominal_ohe = ohe_encoder.transform(cat_data)
nominal_ohe

array([[0., 1., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 0., 1.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 1.]])

In [55]:
# view in DataFrame to understand
pd.DataFrame(nominal_ohe, columns = ohe_encoder.get_feature_names(cat_data.columns))

,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
998,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
999,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1000,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1001,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [57]:
scaler = StandardScaler()

In [61]:
# create tuple of OHE and scaler
ohe_tuple = (ohe_encoder, cat_selector)
scaler_tuple = (scaler, num_selector)
#instantiate column transformer
col_transformer = make_column_transformer(ohe_tuple, scaler_tuple)

In [63]:
#instantiate model
reg = LinearRegression()
pipe = make_pipeline(col_transformer, reg)
#fit on training data
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fef723cc6d0>),
                                                 ('standardscaler',
                                                  StandardScaler

In [65]:
# Evaluate based on training set
train_score = pipe.score(X_train, y_train)
print(train_score)

0.7447321466930588


In [66]:
#evaluate based on test set
test_score = pipe.score(X_test, y_test)
print(test_score)

0.7665551111884477
